In [114]:
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/karinahou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [131]:
yelp_reviews = pd.read_csv("../Data/yelp_review.csv", nrows = 100000)
yelp_business = pd.read_csv("../Data/yelp_business.csv")

In [117]:
yelp_reviews.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [132]:
yelp_business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [119]:
yelp_reviews.text.head()

0    Super simple place but amazing nonetheless. It...
1    Small unassuming place that changes their menu...
2    Lester's is located in a beautiful neighborhoo...
3    Love coming here. Yes the place always needs t...
4    Had their chocolate almond croissant and it wa...
Name: text, dtype: object

In [120]:
remove_punctuation = yelp_reviews['text'].str.replace('[^\w\s]','')

In [121]:
remove_punctuation.head()

0    Super simple place but amazing nonetheless Its...
1    Small unassuming place that changes their menu...
2    Lesters is located in a beautiful neighborhood...
3    Love coming here Yes the place always needs th...
4    Had their chocolate almond croissant and it wa...
Name: text, dtype: object

In [122]:
lowercase = remove_punctuation.str.lower()

In [123]:
lowercase.head()

0    super simple place but amazing nonetheless its...
1    small unassuming place that changes their menu...
2    lesters is located in a beautiful neighborhood...
3    love coming here yes the place always needs th...
4    had their chocolate almond croissant and it wa...
Name: text, dtype: object

In [124]:
split_white = lowercase.str.split()

In [125]:
split_white.head()

0    [super, simple, place, but, amazing, nonethele...
1    [small, unassuming, place, that, changes, thei...
2    [lesters, is, located, in, a, beautiful, neigh...
3    [love, coming, here, yes, the, place, always, ...
4    [had, their, chocolate, almond, croissant, and...
Name: text, dtype: object

In [126]:
stop = stopwords.words('english')
stop_words = split_white.apply(lambda x: [item for item in x if item not in stop])
stop_words.head()

0    [super, simple, place, amazing, nonetheless, a...
1    [small, unassuming, place, changes, menu, ever...
2    [lesters, located, beautiful, neighborhood, si...
3    [love, coming, yes, place, always, needs, floo...
4    [chocolate, almond, croissant, amazing, light,...
Name: text, dtype: object

In [127]:
stemmer = SnowballStemmer("english")
yelp_reviews['cleaned_text'] = stop_words.apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.

In [129]:
yelp_reviews['cleaned_text'].head()

0    [super, simpl, place, amaz, nonetheless, aroun...
1    [small, unassum, place, chang, menu, everi, of...
2    [lester, locat, beauti, neighborhood, sinc, 19...
3    [love, come, yes, place, alway, need, floor, s...
4    [chocol, almond, croissant, amaz, light, butte...
Name: cleaned_text, dtype: object

In [128]:
yelp_reviews.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,cleaned_text
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0,"[super, simpl, place, amaz, nonetheless, aroun..."
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0,"[small, unassum, place, chang, menu, everi, of..."
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0,"[lester, locat, beauti, neighborhood, sinc, 19..."
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0,"[love, come, yes, place, alway, need, floor, s..."
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0,"[chocol, almond, croissant, amaz, light, butte..."
